<a href="https://colab.research.google.com/github/aithaprasad/Bantu_Language_Modeling/blob/main/Bantu_Tweak_Baby_Names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
from keras.layers import GRU 

In [2]:
paths = ['sw-train.txt']
single_string = ''
for filename in paths:
  with open(filename, "r", encoding='utf-8') as f:
    x = f.read()
  single_string += x[:-1] 

In [3]:
len(single_string)

39261061

In [4]:
vocab = set([c for c in single_string])
vocab.add('<PAD>')
encoder = dict((c,i) for i,c in enumerate(vocab))
decoder = dict((i,c) for i,c in enumerate(vocab))

In [ ]:
from tensorflow.keras.utils import to_categorical
X = []
y = []
inputlen = 10

Xenc = [encoder['<PAD>']]*inputlen
for c in single_string:
    X.append(Xenc.copy())
    y.append(encoder[c])
    Xenc.pop(0)
    Xenc.append(encoder[c])
X.append(Xenc.copy())
y.append(encoder['<PAD>'])
    
X = np.array(X)
y = to_categorical(y, num_classes=len(vocab))

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.1, random_state = 42)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
emb_dim = 10
model = Sequential()
model.add(Embedding(input_dim=len(vocab), output_dim=emb_dim, input_length=inputlen))
model.add(GRU(30, recurrent_dropout=0.1, dropout=0.2))
#model.add(Dense(vocab, activation='softmax'))
model.add(Flatten())
model.add(Dense(40))
model.add(Dense(len(vocab), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 10)            320       
                                                                 
 flatten (Flatten)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 40)                4040      
                                                                 
 dense_1 (Dense)             (None, 32)                1312      
                                                                 
Total params: 5,672
Trainable params: 5,672
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(X_train, y_train, batch_size=16384, epochs=50, validation_data=(X_dev, y_dev),verbose=2)

Epoch 1/50
34/34 - 2s - loss: 3.3790 - val_loss: 3.1988 - 2s/epoch - 66ms/step
Epoch 2/50
34/34 - 1s - loss: 2.9722 - val_loss: 2.8158 - 1s/epoch - 37ms/step
Epoch 3/50
34/34 - 1s - loss: 2.7383 - val_loss: 2.6235 - 1s/epoch - 36ms/step
Epoch 4/50
34/34 - 1s - loss: 2.5197 - val_loss: 2.4032 - 1s/epoch - 37ms/step
Epoch 5/50
34/34 - 1s - loss: 2.3413 - val_loss: 2.2796 - 1s/epoch - 37ms/step
Epoch 6/50
34/34 - 1s - loss: 2.2464 - val_loss: 2.2051 - 1s/epoch - 38ms/step
Epoch 7/50
34/34 - 1s - loss: 2.1764 - val_loss: 2.1367 - 1s/epoch - 37ms/step
Epoch 8/50
34/34 - 1s - loss: 2.1085 - val_loss: 2.0696 - 1s/epoch - 36ms/step
Epoch 9/50
34/34 - 1s - loss: 2.0460 - val_loss: 2.0129 - 1s/epoch - 43ms/step
Epoch 10/50
34/34 - 1s - loss: 1.9968 - val_loss: 1.9710 - 1s/epoch - 36ms/step
Epoch 11/50
34/34 - 1s - loss: 1.9611 - val_loss: 1.9405 - 1s/epoch - 36ms/step
Epoch 12/50
34/34 - 1s - loss: 1.9343 - val_loss: 1.9170 - 1s/epoch - 36ms/step
Epoch 13/50
34/34 - 1s - loss: 1.9131 - val_loss:

In [23]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [36]:
def generate(temperature=1.0):
    answer = ''
    Xout = [encoder['<PAD>']]*inputlen
    while True:
        ydist = model.predict(np.array([Xout]), verbose=0)
        #print(ydist)
        nextchar_index = sample(ydist[0],temperature)
        nextchar = decoder[nextchar_index]
        if nextchar == '<PAD>' or len(answer)>10:
            break
        answer += nextchar
        Xout.append(nextchar_index)
        Xout.pop(0)
    return answer
    

In [37]:
def generate_new(temperature=1.0):
    answer = generate(temperature)
    return answer

In [38]:
temp = generate_new(0.5)

In [39]:
print(temp)
print(len(temp))

mbali chiny
11


In [60]:
def anything_goes(lang, c, history):
  Xout = [encoder['<PAD>']]*inputlen
  history_len = len(history)
  prob_of_c = 0
  start = 0
  while start != history_len:
    Xout.append(encoder[history[start]])
    Xout.pop(0)
    start += 1
  ydist = model.predict(np.array([Xout]), verbose=0)
  #nextchar_index = sample(ydist[0])
  #nextchar = decoder[nextchar_index]
  #print(nextchar)
  #print(sum(ydist[0]))
  return ydist[0][encoder[c]]

In [61]:
print(anything_goes('cwe', 'c', []))

0.47961214


In [64]:
from math import log2
def evaluate_one(lang):
  testfile = open(lang+'-test.txt', 'r')
  max_history = 100
  history = []
  loss_anything_goes = 0
  count = 0
  while True:
    c = testfile.read(1)
    if not c:
      break
    count += 1
    loss_anything_goes -= log2(anything_goes(lang, c, history))
    if len(history) == max_history:
      history.pop(0)
    history.append(c)
    print(loss_anything_goes/count)
  return loss_anything_goes/count

In [ ]:
print(evaluate_one('cwe'))

Streaming output truncated to the last 5000 lines.
2.5461695700913034
2.5461592407368494
2.5461289747905522
2.5461018730842873
2.546129112457778
2.546082032291802
2.546150552231323
2.5461020163110866
2.5461035929688918
2.5461274658905015
2.5461321150721665
2.5461721411039067
2.546167159857709
2.546146876843674
2.5461129881144653
2.546071948376601
2.5460391956187785
2.546023279462949
2.546079487137695
2.5461230031660564
2.546150909895805
2.5461349519035563
2.5461356111274
2.54612043459811
2.546095173311057
2.54609820107571
2.546074153772017
2.546053568707865
2.5460312771569362
2.5459996650325936
2.5459591075911225
2.5459298559500056
2.545904720593788
2.545902227878951
2.545899479739203
2.545859817738234
2.545821200570249
2.545846555706317
2.545841383654055
2.545792840288905
2.545753011150749
2.5457605322782872
2.5457385695994863
2.5457346541493533
2.5456989103590315
2.545651255539342
2.5456486253445876
2.5456284091402286
2.545624104314072
2.545575571147127
2.545533363187259
2.5455141082